In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.12.2 requires fsspec==2023.12.2, but you have fsspec 2023.10.0 which is incompatible.


In [2]:
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 560787456 || trainable%: 0.2804741766549072


In [6]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

In [7]:
!pip install wandb -qU

In [8]:
# Log in to your W&B account
import wandb

In [9]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc


True

In [10]:
import wandb  # Import the Weights & Biases library

import random

# # Launch 5 simulated experiments
# total_runs = 1
# for run in range(total_runs):
#   # 🐝 1️⃣ Start a new run to track this script
#   wandb.init(
#       # Set the project where this run will be logged
#       project="LLM", 
#       # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
#       name=f"experiment_{run}", 
#       # Track hyperparameters and run metadata
#       config={
#       "learning_rate": 2e-4,
#       "architecture": "CNN",
#       "dataset": "CIFAR-100",
#       "epochs": 10,
#       })
  
#   # This simple block simulates a training loop logging metrics
#   epochs = 10
#   offset = random.random() / 5
#   for epoch in range(2, epochs):
#       acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#       loss = 2 ** -epoch + random.random() / epoch + offset
      
#       # 🐝 2️⃣ Log metrics from your script to W&B
#       wandb.log({"acc": acc, "loss": loss})
      
#   # Mark the run as finished
#   wandb.finish()



tokenizer.pad_token = tokenizer.eos_token

# Create Transformers Trainer with logging enabled
trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=50,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,  # Log metrics every epoch
        output_dir='outputs',
        # Enable W&B logging
        report_to="wandb",
        overwrite_output_dir=True  # Ensure a unique output directory for each run
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False

# Train the model with W&B logging
trainer.train()

# Finish the W&B run (optional, but recommended)
wandb.finish()


wandb: Currently logged in as: neelghoshal02. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.347100
2,3.342300
3,3.492600
4,3.565900
5,3.575300
6,3.887700
7,3.653600
8,3.396000
9,3.720100
10,3.228300


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.57 GiB. GPU 0 has a total capacty of 14.58 GiB of which 1.33 GiB is free. Process 14055 has 13.24 GiB memory in use. Of the allocated memory 12.81 GiB is allocated by PyTorch, and 307.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF